# Sobre precisión y tolerancia

In [1]:
import numpy as np
import sympy as sp
from numpy import sign

from decimal import * # Este módulo nos permitirá trabajar con una precisión determinada
getcontext()

Context(prec=28, rounding=ROUND_HALF_EVEN, Emin=-999999, Emax=999999, capitals=1, clamp=0, flags=[], traps=[InvalidOperation, DivisionByZero, Overflow])

La precisión por defecto del módulo `Decimal`, como se vio en el notebook de la P1, es 28, nos la guardamos en una variable para poder restaurar la precisión por defecto.

In [2]:
prec_default = getcontext().prec
prec_default

28

A continuación, utilizando la función de bisección del notebook de la P1 (es exactamente la misma pero en este caso especificando valores por defecto de `nmax`, `tol` y `prec`), mostraré el hecho que quiero poner de manifiesto con los ejercicios de la P1.

In [3]:
def biseccion(f, a, b, nmax=100, tol=1e-16, prec=1e-16):
    niter = 0
    cont = True
    exit = ''
    while niter < nmax and cont:
        niter += 1
        c = (a+b)/2
        if abs(f(c)) < prec:    # Si |f(c)|< prec
            exit = 'precision'
            cont = False
        elif sign(f(a)) != sign(f(c)):
            b = c
        else:
            a = c

        if b-a < tol:
            exit = 'tolerancia'
            cont = False

    if exit == 'precision':
        print(f'Posiblemente solución exacta: {c}')
    elif exit == 'tolerancia':
        print(f'Aproximación solicitada: {c}')
    else:
        print('Se llegó al número máximo de iteraciones')
    return c, niter, exit

Definimos la función y los extremos `a` y `b`.

In [4]:
f = lambda x : np.exp(x) - 3
a = Decimal(0)
b = Decimal(2)

Para comparar con la solución exacta, que sabemos que es $\log(3)$, expresamos ese valor con 28 cifras significativas.

In [5]:
sol_exact = sp.log(3)
sol_exact_28 = sp.N(sol_exact, 28) # Solución exacta con 28 cifras significativas
print(f'Solución exacta: {sol_exact_28}')

Solución exacta: 1.098612288668109691395245237


Para que tengáis claro a lo que me refiero con cifras significativas, son básicamente los primeros dígitos no nulos. Para verlo algo mejor, mirad el valor de $\log(3)$ con diferentes números de cifras significativas: 

In [6]:
print(sp.N(sp.log(3), 1) )# 1 cifra significativa
print(sp.N(sp.log(3), 2) )# 2 cifras significativas
print(sp.N(sp.log(3), 5) )# 5 cifras significativas
print(sp.N(sp.log(3), 10)) # 10 cifras significativas
print(sp.N(sp.log(3), 15)) # 15 cifras significativas
print(sp.N(sp.log(3), 20)) # 20 cifras significativas
print(sp.N(sp.log(3), 25)) # 25 cifras significativas
print(sp.N(sp.log(3), 28)) # 28 cifras significativas

1.
1.1
1.0986
1.098612289
1.09861228866811
1.0986122886681096914
1.098612288668109691395245
1.098612288668109691395245237


Sin embargo, mirad qué ocurre si le pido cifras significativas a $10^{-5}=0.00001$:

In [17]:
print(sp.N(0.00001, 1) )# 1 cifra significativa

1.e-5


La primera cifra significativa (el primér dígito no nulo), es 1.

Si ahora, como pido en los ejercicios cambio la precisión del módulo `Decimal` a 5

In [7]:
getcontext().prec = 5 # Establecemos la precisión a 5 decimales

Observad lo que pasa cuando ejecuto bisección con valores de tolerancia y precisión pequeños

In [8]:
aprox, niter , _ = biseccion(f, a, b, nmax=1e3, tol=1e-16, prec=1e-28)
print(f'Número de iteraciones: {niter}')

Posiblemente solución exacta: 1.0986
Número de iteraciones: 11


El programa en cierto momento ha encontrado un valor de $f(c)$ más pequeño que $10^{-5}$ y lo ha redondeado a $0$, por lo que considera que esa es la solución exacta. Pero es que independientemente de los valores de tolerancia y precisión que le indiques a la función va a pasar lo mismo.

In [9]:
aprox, niter , _ = biseccion(f, a, b, nmax=1e3, tol=1e-16, prec=1e-16)
print(f'Número de iteraciones: {niter}')

Posiblemente solución exacta: 1.0986
Número de iteraciones: 11


In [10]:
aprox, niter , _ = biseccion(f, a, b, nmax=1e3, tol=1e-6, prec=1e-10)
print(f'Número de iteraciones: {niter}')

Posiblemente solución exacta: 1.0986
Número de iteraciones: 11


Salvo que indiques una tolerancia mayor que la propia precisión del sistema, por ejemplo, $10^{-2}$

In [11]:
aprox, niter , _ = biseccion(f, a, b, nmax=1e3, tol=1e-2, prec=1e-10)
print(f'Número de iteraciones: {niter}')

Aproximación solicitada: 1.1016
Número de iteraciones: 8


En este caso sale antes por tolerancia que por precisión, pero evidentemente esta aproximación es muy mala. 

La gracia de este primer apartado es que, haciendo esto, los algoritmos en general deberían ser muy insensibles a los argumentos que se le den en tolerancia y precisión y en general encontrar esa "solución exacta".

Retomemos ahora la precisión por defecto.

In [12]:
getcontext().prec = prec_default # Regresamos a la precisión por defecto (28)

En este caso, el módulo `Decimal` difícilmente va a redondear por sí mismo nada a $0$ porque ahora la precisión es muy pequeña. Por tanto, ahora el programa es muy sensible a los argumentos.

En este primer ejemplo, la precisión que le indico es muy pequeña (de hecho la misma que por defecto), con lo cual difícilmente encontrará una raíz exacta.

In [13]:
aprox, niter , _ = biseccion(f, a, b, nmax=1e3, tol=1e-10, prec=1e-28)
print(f'Valor exacto:\t\t {sol_exact_28}')
print(f'Número de iteraciones: {niter}')


Aproximación solicitada: 1.098612288653384894132614136
Valor exacto:		 1.098612288668109691395245237
Número de iteraciones: 35


Sin embargo comprobad que calcula correctamente las primeras 10 cifras significativas (11 de hecho).

In [14]:
aprox, niter , _ = biseccion(f, a, b, nmax=1e3, tol=1e-16, prec=1e-28)
print(f'Valor exacto:\t\t {sol_exact_28}')
print(f'Número de iteraciones: {niter}')

Aproximación solicitada: 1.098612288668109726597066356
Valor exacto:		 1.098612288668109691395245237
Número de iteraciones: 55


Y en este caso las 16 primeras cifras significativas.

Ahora voy a probar a bajar la precisión.

In [15]:
aprox, niter , _ = biseccion(f, a, b, nmax=1e3, tol=1e-16, prec=1e-10)
print(f'Valor exacto:\t\t {sol_exact_28}')
print(f'Número de iteraciones: {niter}')

Posiblemente solución exacta: 1.098612288653384894132614136
Valor exacto:		 1.098612288668109691395245237
Número de iteraciones: 35


En este caso también calcula la solución con al menos 10 cifras significativas, pero determina que esa solución es exacta porque ha salido por precisión, no por tolerancia.

Otra forma de comprobar el número de cifras significativas que coinciden es calcular la diferencia entre el valor exacto y la aproximación. En este último caso:

In [16]:
abs(aprox- sol_exact_28)

1.472479726263110116745845432e-11

Como podéis comprobar, por una vía o por otra se puede obtener una aproximación tan buena como queramos. El número de iteraciones va a depender mucho del algoritmo, ya que los hay más rápidos y más lentos, pero precisamente es eso lo que quiero que comprobéis. 

Cualquier otra duda que tengáis no dudéis en preguntar. ¡Ánimo!